In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Input, Dropout
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [2]:
def generator():
    gen = Sequential()
    gen.add(Dense(units = 256, input_dim = 100))
    gen.add(LeakyReLU(0.2))
    
    gen.add(Dense(units = 512))
    gen.add(LeakyReLU(0.2))
    
    gen.add(Dense(units = 1024))
    gen.add(LeakyReLU(0.2))
    
    gen.add(Dense(units = 784, activation = 'tanh'))
    gen.compile(loss = 'binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    
    return gen

In [3]:
g = generator()

In [4]:
g.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 784)              

In [5]:
def discriminator():
    dis = Sequential()
    dis.add(Dense(units = 1024, input_dim = 784))
    dis.add(LeakyReLU(0.2))
    
    dis.add(Dense(units = 512))
    dis.add(LeakyReLU(0.2))
    
    dis.add(Dense(units = 256))
    dis.add(LeakyReLU(0.2))
    
    dis.add(Dense(units = 1, activation = 'sigmoid'))
    dis.compile(loss = 'binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    
    return dis

In [6]:
d = discriminator()

In [7]:
d.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [8]:
dis = discriminator()
gen = generator()
dis.trainable = False
gan_input = Input(shape = (100,))
fake_img = gen(gan_input)
gan_output = dis(fake_img)
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer = Adam(0.0002,0.5))

In [13]:
def plot_images(epoch, gen, examples = 100, dim = (10,10)):
    noise = np.random.normal(0,1,[examples, examples])
    gen_images = gen.predict(noise)
    gen_images = gen_images.reshape(100,28,28)
    plt.figure(figsize=dim)
    for i in range(gen_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(gen_images[i])
        plt.axis('off')
    plt.savefig('Gan_imgs/img_{}.png'.format(epoch))

In [14]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [15]:
x_train = (x_train.astype(np.float32) - 127.5) / 127.5

In [16]:
gen_images.shape

NameError: name 'gen_images' is not defined

In [ ]:
epochs = 100
batch_size = 128
n = x_train.shape[0]
batch_steps = int(n / batch_size)
for i in range(epochs):
    for j in range(batch_steps):
        noise = np.random.normal(0,1,[batch_size,100])
        gen_images = gen.predict(noise)

        img_batch = x_train[np.random.randint(0,n,size=batch_size)]
        img_batch = img_batch.reshape(batch_size, -1)
        X = np.concatenate([img_batch, gen_images])
        y_dis = np.zeros(2 * batch_size)
        y_dis[:batch_size] = 0.9
        d_loss = dis.train_on_batch(X,y_dis)

        y_gen = np.ones(batch_size)
        g_loss = gan.train_on_batch(noise, y_gen)

    print("Epoch : {}, Dis Loss : {}, Gen Loss : {}".format(i, d_loss, g_loss))
    plot_images(i, gen)

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch : 0, Dis Loss : 0.4388490915298462, Gen Loss : 2.32437801361084
Epoch : 1, Dis Loss : 0.4419821500778198, Gen Loss : 1.712998867034912
Epoch : 2, Dis Loss : 0.37339967489242554, Gen Loss : 1.8492540121078491
Epoch : 3, Dis Loss : 0.445328950881958, Gen Loss : 2.4001450538635254
Epoch : 4, Dis Loss : 0.45715463161468506, Gen Loss : 1.4929814338684082
Epoch : 5, Dis Loss : 0.5019633769989014, Gen Loss : 1.4350717067718506
Epoch : 6, Dis Loss : 0.4970414638519287, Gen Loss : 1.4987508058547974
Epoch : 7, Dis Loss : 0.4848541021347046, Gen Loss : 1.8337266445159912
Epoch : 8, Dis Loss : 0.546810507774353, Gen Loss : 1.4291768074035645
Epoch : 9, Dis Loss : 0.5047929883003235, Gen Loss : 1.4098957777023315
Epoch : 10, Dis Loss : 0.5098594427108765, Gen Loss : 1.6404244899749756
Epoch : 11, Dis Loss : 0.5573514699935913, Gen Loss : 1.7523643970489502
Epoch : 12, Dis Loss : 0.5148091912269592, Gen Loss : 1.5392301082611084
Epoch : 13, Dis Loss : 0.48987647891044617, Gen Loss : 1.9405006

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


Epoch : 21, Dis Loss : 0.5225095152854919, Gen Loss : 1.5598052740097046
Epoch : 22, Dis Loss : 0.5245570540428162, Gen Loss : 1.7110059261322021
Epoch : 23, Dis Loss : 0.5542501211166382, Gen Loss : 1.6015143394470215
Epoch : 24, Dis Loss : 0.5497171878814697, Gen Loss : 1.5199925899505615
Epoch : 25, Dis Loss : 0.5151606798171997, Gen Loss : 1.5620191097259521
Epoch : 26, Dis Loss : 0.5180884599685669, Gen Loss : 1.5546183586120605
Epoch : 27, Dis Loss : 0.5555636882781982, Gen Loss : 1.580253005027771
Epoch : 28, Dis Loss : 0.5358455181121826, Gen Loss : 1.7288784980773926
Epoch : 29, Dis Loss : 0.5163276195526123, Gen Loss : 1.9409394264221191
Epoch : 30, Dis Loss : 0.5637374520301819, Gen Loss : 1.7372663021087646
Epoch : 31, Dis Loss : 0.5269889831542969, Gen Loss : 1.8274953365325928
Epoch : 32, Dis Loss : 0.5411231517791748, Gen Loss : 1.3707308769226074
Epoch : 33, Dis Loss : 0.5691149830818176, Gen Loss : 2.2213621139526367
Epoch : 34, Dis Loss : 0.5437407493591309, Gen Loss 